## 📦 Step 1: Install Required Libraries

In [ ]:
# Install libraries (takes 2-3 minutes)
!pip install -q transformers datasets accelerate peft bitsandbytes trl huggingface_hub
!pip install -q torch torchvision torchaudio

print("✅ Libraries installed successfully!")

## 🔑 Step 2: Login to Hugging Face

1. Go to https://huggingface.co and create FREE account
2. Go to Settings → Access Tokens → Create new token
3. Copy the token and paste below

In [ ]:
from huggingface_hub import login

# Your Hugging Face token - Get from: https://huggingface.co/settings/tokens
# IMPORTANT: Replace this with YOUR token when running in Colab!
HF_TOKEN = "YOUR_HF_TOKEN_HERE"  # ← Paste your token here

login(token=HF_TOKEN)

print("✅ Logged in to Hugging Face!")

## 📊 Step 3: Load Your Training Data

In [ ]:
import json

# Load training data
with open('jarvis_training_data.json', 'r', encoding='utf-8') as f:
    training_data = json.load(f)

print(f"📊 Model Name: {training_data['model_name']}")
print(f"📊 Total Examples: {training_data['total_examples']}")
print(f"📊 Created By: {training_data['created_by']}")
print()
print("📋 Sample Data:")
print("-" * 50)
sample = training_data['data'][0]
print(f"Question: {sample['instruction'][:100]}...")
print(f"Answer: {sample['output'][:200]}...")

## 🤖 Step 4: Load Base Model

We'll use **TinyLlama** for faster training (good for learning).
Later you can upgrade to Llama 3 or Mistral.

In [ ]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Check GPU
print(f"🖥️ GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🖥️ GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"🖥️ GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# Model name (TinyLlama for faster training)
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# For better results later, use:
# MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"
# MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"

print(f"\n📥 Loading model: {MODEL_NAME}")
print("This may take 2-5 minutes...")

In [ ]:
# Quantization config (reduces memory usage)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Load model
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

print("✅ Model loaded successfully!")

## 🎯 Step 5: Prepare for Fine-Tuning (LoRA)

**LoRA** = Low-Rank Adaptation
- Trains only small parts of the model
- Much faster and uses less memory
- Results are almost as good as full training

In [ ]:
# Prepare model for training
model = prepare_model_for_kbit_training(model)

# LoRA configuration
lora_config = LoraConfig(
    r=16,                      # Rank (higher = more capacity)
    lora_alpha=32,             # Scaling factor
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],  # Which layers to train
    lora_dropout=0.05,         # Dropout for regularization
    bias="none",
    task_type="CAUSAL_LM"
)

# Apply LoRA
model = get_peft_model(model, lora_config)

# Print trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())

print(f"\n📊 Model Statistics:")
print(f"   Total parameters: {total_params:,}")
print(f"   Trainable parameters: {trainable_params:,}")
print(f"   Trainable %: {100 * trainable_params / total_params:.2f}%")
print("\n✅ Model ready for fine-tuning!")

## 📝 Step 6: Prepare Training Dataset

In [ ]:
from datasets import Dataset

def format_prompt(example):
    """Format data for training"""
    prompt = f"""### Instruction:
{example['instruction']}

### Response:
{example['output']}"""
    return {"text": prompt}

# Create dataset
data = training_data['data']
dataset = Dataset.from_list(data)
dataset = dataset.map(format_prompt)

print(f"✅ Dataset created with {len(dataset)} examples")
print()
print("📋 Sample formatted prompt:")
print("-" * 50)
print(dataset[0]['text'][:500] + "...")

## 🚀 Step 7: Train Your JARVIS AI!

This is where the magic happens! ✨

Training time: ~10-30 minutes depending on data size

In [ ]:
from trl import SFTTrainer

# Your Hugging Face username
HF_USERNAME = "aijarvis2025"

# Output model name - Your AI model!
OUTPUT_MODEL = f"{HF_USERNAME}/jarvis-edu-ai"

print(f"📤 Your model will be saved to: https://huggingface.co/{OUTPUT_MODEL}")

# Training arguments
training_args = TrainingArguments(
    output_dir="./jarvis-model",
    num_train_epochs=3,              # Number of times to go through data
    per_device_train_batch_size=4,   # Samples per batch
    gradient_accumulation_steps=4,   # Accumulate gradients
    warmup_steps=10,                 # Warmup learning rate
    learning_rate=2e-4,              # Learning rate
    bf16=True,                       # Use bfloat16 precision (better for newer GPUs)
    logging_steps=10,                # Log every N steps
    save_strategy="epoch",           # Save after each epoch
    optim="paged_adamw_8bit",        # Optimizer
    report_to="none"                 # Don't report to wandb
)

# Create trainer (minimal version for newest TRL API)
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    args=training_args
)

print("🚀 Ready to train!")
print("Run the next cell to start training...")
print("This will take 10-30 minutes. Go grab a coffee! ☕")

In [ ]:
# Train the model!
trainer.train()

print()
print("🎉 Training complete!")
print("Your JARVIS AI has learned from your data!")

## 💾 Step 8: Save Your Model

In [ ]:
# Save locally first
trainer.save_model("./jarvis-final")
tokenizer.save_pretrained("./jarvis-final")

print("✅ Model saved locally!")

## ☁️ Step 9: Upload to Hugging Face (FREE Hosting!)

In [ ]:
from huggingface_hub import HfApi

# Create model card
model_card = f"""---
license: apache-2.0
language:
- en
tags:
- education
- jarvis
- ai-tutor
---

# 🧠 JARVIS Educational AI

## Created by: Vishal

This is a fine-tuned language model specialized for educational assistance.

### What JARVIS Can Do:
- 💻 Programming help (Python, JavaScript, Java, etc.)
- 📊 Data Structures & Algorithms explanations
- 🌐 Web Development guidance
- 💼 Career advice for IT freshers
- 📚 Study tips and learning strategies

### Usage:
```python
from transformers import pipeline

pipe = pipeline("text-generation", model="{OUTPUT_MODEL}")
response = pipe("What is Python?")[0]['generated_text']
print(response)
```

### Training:
- Base Model: TinyLlama-1.1B-Chat
- Method: LoRA Fine-tuning
- Data: Custom educational Q&A dataset

Built for 30,000+ students! 🎓
"""

# Save model card
with open("./jarvis-final/README.md", "w") as f:
    f.write(model_card)

print("✅ Model card created!")

In [ ]:
# Upload to Hugging Face
print(f"📤 Uploading to Hugging Face: {OUTPUT_MODEL}")
print("This may take a few minutes...")

model.push_to_hub(OUTPUT_MODEL)
tokenizer.push_to_hub(OUTPUT_MODEL)

print()
print("🎉🎉🎉 SUCCESS! 🎉🎉🎉")
print()
print(f"Your JARVIS AI is now live at:")
print(f"https://huggingface.co/{OUTPUT_MODEL}")
print()
print("You can now use this model in your website!")

## 🧪 Step 10: Test Your JARVIS AI!

In [ ]:
def ask_jarvis(question):
    """Ask JARVIS a question"""
    prompt = f"""### Instruction:
{question}

### Response:
"""
    
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        temperature=0.7,
        do_sample=True,
        top_p=0.9
    )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.split("### Response:")[-1].strip()

# Test questions
print("🧪 Testing JARVIS AI...")
print("=" * 50)

questions = [
    "What is Python?",
    "Explain loops in programming",
    "How to get a job as fresher?"
]

for q in questions:
    print(f"\n❓ Question: {q}")
    print(f"🤖 JARVIS: {ask_jarvis(q)[:500]}...")
    print("-" * 50)

---

# 🎉 Congratulations!

## You just built your own AI model!

### What You Achieved:
- ✅ Downloaded a pre-trained language model
- ✅ Fine-tuned it on YOUR educational data
- ✅ Created YOUR OWN JARVIS AI
- ✅ Uploaded to Hugging Face (FREE hosting)

### Next Steps:
1. **Add More Training Data** - More data = Better AI
2. **Upgrade Base Model** - Try Llama 3 or Mistral
3. **Connect to Website** - Use your model in JARVIS
4. **Add RAG System** - Give JARVIS your knowledge base

---

### You're now an AI Developer! 🧠🚀